# 第6讲：非线性优化



## 6.1 状态估计问题

### 6.1.1 最大后验与最大似然

SLAM模型由运动方程和观测方程组成：

$$\begin{cases}
\mathbf{x}_{k} = f(\mathbf{x}_{k - 1}, \mathbf{u}_{k}) + \mathbf{w}_{k} \\
\mathbf{z}_{k, j} = h(\mathbf{y}_{j}, \mathbf{x}_{k}) + \mathbf{v}_{k, j} \\
\end{cases} \tag {6-1}$$

其中，$\mathbf{x}_{k}$为相机位姿，可以通过变换矩阵$\mathbf{T}_{k}$或李代数$\exp(\mathbf{\xi}_{k}^{\wedge})$计算。观测方程使用针孔相机模型描述。

* 观测方程

假设机器人在$\mathbf{x}_{k}$处对路标$\mathbf{y}_{j}$进行观测，其在图像中的像素坐标为$\mathbf{z}_{k, j}$，则观测方程表示为：

$$s \mathbf{z}_{k, j} = \mathbf{K} \exp(\mathbf{\xi}_{k}^{\wedge}) \mathbf{y}_{j} \tag {6-2}$$

其中，$\mathbf{K}$为相机内参，$s$表示$\mathbf{y}_{j}$到相机的距离（相机坐标系下$Z$值），$\mathbf{z}_{k, j}$和$\mathbf{y}_{j}$用齐次坐标表示。考虑加噪数据，假设噪声项$\mathbf{w}_{k}$、$\mathbf{v}_{k, j}$满足零均值高斯分布：

$$\mathbf{w}_{k} \sim N(0, \mathbf{R}_{k}), \mathbf{v}_{k, j} \sim N(0, \mathbf{Q}_{k, j}) \tag {6-3}$$

在噪声影响下，通过观测数据$\mathbf{z}$和$\mathbf{u}$推断位姿$\mathbf{x}$和地图$\mathbf{y}$（以及概率分布）是状态估计问题。由于SLAM过程中，这些数据是时间序列，过去的研究者们主要使用滤波器（扩展卡尔曼滤波器，EKF）对其求解。卡尔曼滤波器只关心当前时刻的状态估计$\mathbf{x}_{k}$，对历史状态则不多考虑；相对的，近年来普遍采用的非线性优化方法，使用所有时刻数据进行状态估计，通常认为优于传统滤波器，成为当前视觉SLAM主流方法。

从概率学角度出发，非线性优化中，将全部待估计变量视为一个“状态变量”：

$$\mathbf{x} = \{ \mathbf{x}_{1}, \dots, \mathbf{x}_{N}, \mathbf{y}_{1}, \dots, \mathbf{y}_{M} \}$$

对机器人状态的估计，即是在已知输入数据$\mathbf{y}$和观测数据$\mathbf{z}$的条件下，计算状态$\mathbf{x}$的条件概率分布：

$$P(\mathbf{x} | \mathbf{z}, \mathbf{u}) \tag {6-4}$$

当机器人未装备测量运动的传感器时，只需考虑观测方程，即估计条件概率$P(\mathbf{x} | \mathbf{z}, \mathbf{u})$的
分布。如果忽略图像的时序关系，将它们视为彼此无关的图片，则该问题称为SfM（structure from motion）问题，即从多张图像中重建三维空间结构。根据贝叶斯法则，

$$P(\mathbf{x} | \mathbf{z})
= \frac{P(\mathbf{z} | \mathbf{x}) P(\mathbf{x})}{P(\mathbf{z})}
\propto P(\mathbf{z} | \mathbf{x}) P(\mathbf{x}) \tag {6-5}$$

其中，称$P(\mathbf{x} | \mathbf{z})$为**后验概率**、$P(\mathbf{z} | \mathbf{x})$为**似然**，$P(\mathbf{x})$为**先验**。

后验分布通常难以直接求解，一般采用点估计方式，求解状态的最优估计（在该状态下后验概率最大，即最大后验估计（maximize a posterior，MAP））：

$$\mathbf{x}_{\text{MAP}^{\ast}} = \argmax P(\mathbf{x} | \mathbf{z}) = \argmax P(\mathbf{z} | \mathbf{x}) P(\mathbf{x}) \tag {6-6}$$

*最大后验概率相当于最大化似然和先验的乘积*。当缺少机器人位姿先验信息时，可以求解状态$\mathbf{x}$的最大似然估计（maximize likelihood estimation，MLE）：

$$\mathbf{x}_{\text{MLE}^{\ast}} = \argmax P(\mathbf{z} | \mathbf{x}) \tag {6-7}$$

似然是指“在当前位姿下，可能产生怎样的观测数据”。由于只有观测数据已知，*最大似然估计表示：在何种状态下，最可能产生当前的观测数据*。